In [1]:
from SAES import SAES

saes = SAES()
def apply_tampering(ciphertext, bits_to_flip, total_bits=64):
    """这个函数将从第一个块的开始位置翻转指定数量的比特。"""
    mask = 0
    for i in range(bits_to_flip):
        mask <<= 1
        mask |= 1

    # 确保mask移动到ciphertext的前部
    mask <<= total_bits - bits_to_flip
    return ciphertext ^ mask


def test_cbc_tampering(saes, plaintext, key, iv):
    # 加密原始明文
    original_ciphertext = saes.CBC_encrypt(plaintext, key, iv)
    print("Original ciphertext:", hex(original_ciphertext))

    total_bits = len(bin(original_ciphertext)) - 2  # 计算密文总比特数

    # 定义篡改列表
    tamperings = [4, 8, 12, 16, 32]  # 32比特意味着两个块

    for tampering_bits in tamperings:
        print(f"\nTampering {tampering_bits} bits...")
        tampered_ciphertext = apply_tampering(original_ciphertext, tampering_bits)
        print("Tampered ciphertext:", hex(tampered_ciphertext))

        # 解密篡改后的密文
        decrypted_tampered = saes.CBC_decrypt(tampered_ciphertext, key, iv)
        print("Decrypted tampered ciphertext:", hex(decrypted_tampered))

key = 0x1234
iv = 0x5678
plaintext = 0x9abc9abcdef01234
test_cbc_tampering(saes, plaintext, key, iv)

Original ciphertext: 0xf2cf10e25b8dfd0a

Tampering 4 bits...
Tampered ciphertext: 0x2cf10e25b8dfd0a
Decrypted tampered ciphertext: 0x6abe6abcdef01234

Tampering 8 bits...
Tampered ciphertext: 0xdcf10e25b8dfd0a
Decrypted tampered ciphertext: 0x675e65bcdef01234

Tampering 12 bits...
Tampered ciphertext: 0xd3f10e25b8dfd0a
Decrypted tampered ciphertext: 0x6d7e654cdef01234

Tampering 16 bits...
Tampered ciphertext: 0xd3010e25b8dfd0a
Decrypted tampered ciphertext: 0x4d716543def01234

Tampering 32 bits...
Tampered ciphertext: 0xd30ef1d5b8dfd0a
Decrypted tampered ciphertext: 0x4d719659210f1234


**Key Used for Encryption**: `0x1234`
**Initialization Vector (IV)**: `0x5678`

| Tampering | Original Ciphertext | Tampered Ciphertext | Decrypted Tampered Ciphertext |
| --------- | ------------------- | ------------------- | ----------------------------- |
| None      | 0xf2cf10e25b8dfd0a  | N/A                 | 0x9abc9abcdef01234            |
| 4 bits    | 0xf2cf10e25b8dfd0a  | 0x02cf10e25b8dfd0a  | 0x6abe6abcdef01234            |
| 8 bits    | 0xf2cf10e25b8dfd0a  | 0x0dcf10e25b8dfd0a  | 0x675e65bcdef01234            |
| 12 bits   | 0xf2cf10e25b8dfd0a  | 0x0d3f10e25b8dfd0a  | 0x6d7e654cdef01234            |
| 16 bits   | 0xf2cf10e25b8dfd0a  | 0x0d3010e25b8dfd0a  | 0x4d716543def01234            |
| 32 bits   | 0xf2cf10e25b8dfd0a  | 0x0d30ef1d5b8dfd0a  | 0x4d719659210f1234            |

分析如下：

1. **错误传播特性**：从结果中，我们可以看到CBC模式的一个显著特点，即一个加密块的篡改会影响到解密的该块和下一个块。这称为错误传播特性。比如当我们篡改第一个块时，解密的第一个块和第二块都受到了影响。

2. **一致性的缺失**：尽管错误传播到后续块，但篡改加密的块并不能以有意义的方式影响解密的块。换句话说，一个小小的篡改（例如4比特）可能会导致完全不同的解密输出，而不是仅仅4比特的差异。这也是为什么篡改加密消息是危险的，因为它会导致完全不可预测的输出。

3. **影响的持续性**：当我们篡改32位时，尽管第一个块和第二块都受到了影响，但从第三块开始，解密的输出与原始明文相同。这证明了CBC模式中错误传播的“有限”特性，即一个错误只会影响其本身和下一个块。

4. **安全性与篡改检测**：CBC模式本身并不提供明文的完整性或篡改检测。从结果中，我们可以看到，即使对加密文本进行了小小的篡改，也可能不容易被接收者察觉，除非他们期望的解密输出与实际的解密输出存在显著差异。

5. **篡改与噪声**：篡改在真实世界中可能被视为噪声或数据损坏。CBC模式对于随机噪声具有某种“弹性”，因为噪声只影响两个块，而不是整个消息。

总之，CBC模式提供了一种方式，使篡改的加密消息在解密时产生不可预测的输出，并且篡改会传播到后续的块，但其影响是有限的。为了提高安全性，应该考虑使用其他机制来确保数据的完整性和篡改检测。

